# 📰 Fake News Detection using Machine Learning
This project builds a machine learning model to detect fake news using text processing and logistic regression.

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
    ```
    1: Fake news
    0: real News
    ```

**Importing the Dependencies**

In [36]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**Downloading The Stopwords**

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

**Data Pre-processing**

In [39]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/drive/MyDrive/Machine Learning/004 Fake News Prediction using Machine Learning with Python/train.csv')

In [40]:
news_dataset.shape

(20800, 5)

In [41]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [42]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [43]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [44]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [45]:
# merging the author name and news title
news_dataset['content'] = news_dataset['title'] + ' ' + news_dataset['author']

The news_dataset['text'] column is not included here, as it takes too long to execute. However, even without it, the model achieves a very good accuracy score.

In [46]:
news_dataset['content']

,content
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired Consortiumne...
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


**Stemming:**

Stemming is the process of reducing a word to its Root word.

example: actor, actress, acting --> act

In [47]:
port_stem = PorterStemmer()

In [48]:
def stemming(content):
  stemmed_content = re.sub('(^a-zA-Z)',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [49]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [50]:
news_dataset['content']

,content
0,hous dem aide: didn’t even see comey’ letter j...
1,"flynn: hillari clinton, big woman campu - brei..."
2,truth might get fire consortiumnews.com
3,15 civilian kill singl us airstrik identifi je...
4,iranian woman jail fiction unpublish stori wom...
...,...
20795,rapper t.i.: trump ’poster child white suprema...
20796,"n.f.l. playoffs: schedule, matchup odd - new y..."
20797,macy’ said receiv takeov approach hudson’ bay ...
20798,"nato, russia hold parallel exercis balkan alex..."


In [51]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [52]:
X

array(['hous dem aide: didn’t even see comey’ letter jason chaffetz tweet darrel lucu',
       'flynn: hillari clinton, big woman campu - breitbart daniel j. flynn',
       'truth might get fire consortiumnews.com', ...,
       'macy’ said receiv takeov approach hudson’ bay - new york time michael j. de la merc rachel abram',
       'nato, russia hold parallel exercis balkan alex ansari',
       'keep f-35 aliv david swanson'], dtype=object)

In [53]:
Y

array([1, 0, 1, ..., 0, 1, 1])

In [54]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
vectorized_X = vectorizer.transform(X)

In [55]:
vectorized_X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 218021 stored elements and shape (20800, 21822)>

**Splitting the dataset to training & test data**

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(vectorized_X,Y, test_size = 0.2, stratify = Y, random_state = 2)

**Training the Model: Logistic Regression**

In [57]:
model = LogisticRegression()

In [58]:
model.fit(X_train, Y_train)

LogisticRegression()

**Evaluation**

Accuracy score

In [59]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('The accuracy of the model on training data is: {:0.2f}%'.format(training_data_accuracy*100))

The accuracy of the model on training data is: 98.66%


In [60]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('The accuracy of the model on test data is: {:0.2f}%'.format(test_data_accuracy*100))

The accuracy of the model on test data is: 97.74%


**Making a Predictive System**

In [61]:
def predictive_system(n):
  prediction = model.predict(X_test[n])
  if prediction == 0:
    print('The news is Real')
  else:
    print('The news is Fake')
  if prediction == Y_test[n]:
    print('Congratulations! Your model provided accurate result.')
  else:
    print('Warning! The model has made a mistake.')

In [62]:
m = int(input('Enter a number between 0 and {}: '.format(Y_test.shape[0]-1)))
predictive_system(m)

Enter a number between 0 and 4159: 519
The news is Fake
Congratulations! Your model provided accurate result.
